In [1]:
import datasets
from datasets import Dataset, load_dataset, load_from_disk
from torch.utils.data import DataLoader
from transformers import TrainingArguments, Trainer
from transformers import Wav2Vec2CTCTokenizer, HubertConfig, HubertModel, Wav2Vec2FeatureExtractor, Wav2Vec2Processor

/home/kaisar_dauletbek/anaconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-14 17:17:26.896842: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 17:17:27.563432: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-14 17:17:27.563504: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvi

In [2]:
dataset = load_from_disk('/raid/kaisar_dauletbek/Kazakh-Hubert/dataset')

In [3]:
dataset

Dataset({
    features: ['path', 'transcript', 'split', 'input_ids', 'attention_mask', 'speech'],
    num_rows: 18036
})

In [4]:
dataset['speech'][0].shape

In [6]:
dataloader = DataLoader(dataset, shuffle=True, batch_size=16)

In [11]:
# Initialize a new tokenizer
tokenizer = Wav2Vec2CTCTokenizer("/raid/kaisar_dauletbek/Kazakh-Hubert/vocab.json", unk_token="<unk>", pad_token="<pad>", word_delimiter_token=" ")


# Train the tokenizer on your data
# tokenizer.train(files=dataset["path"])

# Initialize a new model configuration
config = HubertConfig()

# Initialize a new model with the configuration
model = HubertModel(config)

# Initialize a new feature extractor
feature_extractor = Wav2Vec2FeatureExtractor()

# Combine the tokenizer and feature extractor into a processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [8]:
training_args = TrainingArguments(
    output_dir="/raid/kaisar_dauletbek/Kazakh-Hubert",
    group_by_length=True,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=30,
    fp16=True,
    save_steps=500,
    eval_steps=500,
    logging_steps=500,
    learning_rate=1e-4,
    warmup_steps=500,
    save_total_limit=2,
)

In [12]:
trainer = Trainer(
    model=model,
    # data_collator=data_collator,
    args=training_args,
    # compute_metrics=compute_metrics,
    train_dataset=dataset,
)

trainer.train()

Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `HubertModel.forward` and have been ignored: split, speech, path, transcript, input_ids. If split, speech, path, transcript, input_ids are not expected by `HubertModel.forward`,  you can safely ignore this message.


KeyboardInterrupt: 

In [14]:
import torch
torch.cuda.device_count()

8

In [ ]:
dataset['speech'][0].shape